In [268]:
#pip install pandas pyodbc sqlalchemy


In [269]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.engine import URL

# Step 1: Define connection info
connection_url = URL.create(
    "mssql+pyodbc",
    username="sa",
    password="Hasini@07",
    host="DESKTOP-1M0C5L8\\SQLEXPRESS01",
    database="pipelines_day2",
    query={"driver": "ODBC Driver 17 for SQL Server"}
)

# Step 2: Create engine
engine = create_engine(connection_url)

# Step 3: Define CSV to table mapping
csv_files = {
    'us_customer_data.csv': 'customers',
    'order_data.csv': 'orders'
}

# Step 4: Upload each CSV file to corresponding SQL table
for csv_file, table_name in csv_files.items():
    df = pd.read_csv(csv_file)
    df.to_sql(table_name, con=engine, if_exists='replace', index=False)
    print(f" Uploaded {csv_file} as table `{table_name}`.")


 Uploaded us_customer_data.csv as table `customers`.
 Uploaded order_data.csv as table `orders`.


In [270]:
import pandas as pd

# Extract customers table into a DataFrame
customers_df = pd.read_sql("SELECT * FROM customers", con=engine)
print("Extracted customers table:")
customers_df.head()


Extracted customers table:


,customer_id,name,email,phone,address,registration_date,loyalty_status
0,1,Michelle Kidd,vayala@example.net,619-723-4258,"USNS Santiago, FPO AE 80872",25-01-2025,Gold
1,2,Brad Newton,taylorcatherine@example.net,537-674-1158,"38783 Oliver Street, West Kristenborough, MT 9...",13-07-2023,Silver
2,3,Larry Torres,dsanchez@example.net,810-256-4505,"6845 Steele Turnpike, West Erikabury, UT 37487",18-08-2023,Bronze
3,4,Kimberly Price,jessicaknight@example.com,423-222-9779,"1631 Alexis Meadows, Lake Amanda, CA 75179",08-12-2024,Gold
4,5,Matthew Phillips,qwilliams@example.com,220-763-3522,"2274 Williams Heights Suite 895, Andersonhaven...",03-02-2024,Gold


In [271]:
# Extract orders table into a DataFrame
orders_df = pd.read_sql("SELECT * FROM orders", con=engine)
print("Extracted orders table:")
orders_df.head()


Extracted orders table:


,order_id,customer_id,order_date,order_amount,order_status,product_category
0,892a07a4-d252-4775-85e0-73077143e1c6,966,2024-11-27,317.64,Cancelled,Home & Garden
1,ae160758-e187-47b2-9350-032f88f55491,345,2023-03-27,645.87,Completed,Home & Garden
2,7c50456e-6123-45cc-aa19-128bef3754d6,503,2024-03-31,880.86,Pending,Clothing
3,c788b56b-3716-4cd9-a827-d4dc401ba00c,385,2023-08-09,876.83,Cancelled,Home & Garden
4,925ab5b1-adb1-4302-a70b-1c2db724e02b,817,2023-04-19,264.53,Pending,Home & Garden


In [272]:
# Merge customers and orders using customer_id
df = pd.merge(customers_df, orders_df, on='customer_id', how='inner')

print("Unified Customer View created. Preview:")
df.head()


Unified Customer View created. Preview:


,customer_id,name,email,phone,address,registration_date,loyalty_status,order_id,order_date,order_amount,order_status,product_category
0,2,Brad Newton,taylorcatherine@example.net,537-674-1158,"38783 Oliver Street, West Kristenborough, MT 9...",13-07-2023,Silver,209c024e-6c32-49fa-acd1-2501bd19c1cd,2023-10-30,486.53,Pending,Toys
1,2,Brad Newton,taylorcatherine@example.net,537-674-1158,"38783 Oliver Street, West Kristenborough, MT 9...",13-07-2023,Silver,3f263091-da18-48a6-8b00-c179e1b1a81a,2024-08-29,781.22,Pending,Books
2,3,Larry Torres,dsanchez@example.net,810-256-4505,"6845 Steele Turnpike, West Erikabury, UT 37487",18-08-2023,Bronze,63d7870e-617e-4f49-ae9b-b893e635e43f,2023-04-05,454.07,Pending,Toys
3,4,Kimberly Price,jessicaknight@example.com,423-222-9779,"1631 Alexis Meadows, Lake Amanda, CA 75179",08-12-2024,Gold,cbe7b5ef-013f-4b4f-9ac4-3b510075080a,2023-08-12,259.07,Cancelled,Electronics
4,4,Kimberly Price,jessicaknight@example.com,423-222-9779,"1631 Alexis Meadows, Lake Amanda, CA 75179",08-12-2024,Gold,21209e05-078a-402d-a450-dd310595e5f0,2024-08-27,117.03,Completed,Electronics


In [273]:
df.isnull().sum()  # Check for null values in the merged DataFrame

customer_id            0
name                   0
email                106
phone                 35
address                0
registration_date      0
loyalty_status         0
order_id               0
order_date             0
order_amount           0
order_status           0
product_category       0
dtype: int64

In [274]:
df.nunique()  # Check for unique values in the merged DataFrame

customer_id           868
name                  859
email                 763
phone                 852
address               868
registration_date     513
loyalty_status          3
order_id             2000
order_date            688
order_amount         1974
order_status            3
product_category        5
dtype: int64

In [275]:
df.shape

(2000, 12)

In [276]:
# Define regex for prefixes and suffixes
prefixes = r'^(Mr\.|Mrs\.|Miss|Ms\.|Dr\.)\s*'
suffixes = r'\s*(Jr\.|Sr\.|II|III|DDS|PhD|MD|MBA|DVM|Esq\.?)$'


# Step 2: Remove prefix from beginning
df['name'] = df['name'].str.replace(prefixes, '', regex=True)

# Step 3: Remove suffix from end
df['name'] = df['name'].str.replace(suffixes, '', regex=True)


In [277]:
print(df)

      customer_id            name                        email         phone  \
0               2     Brad Newton  taylorcatherine@example.net  537-674-1158   
1               2     Brad Newton  taylorcatherine@example.net  537-674-1158   
2               3    Larry Torres         dsanchez@example.net  810-256-4505   
3               4  Kimberly Price    jessicaknight@example.com  423-222-9779   
4               4  Kimberly Price    jessicaknight@example.com  423-222-9779   
...           ...             ...                          ...           ...   
1995          997    Jodi Simpson           eric24@example.org  483-625-2940   
1996          998   Crystal Brown         pshaffer@example.net  390-747-3088   
1997          998   Crystal Brown         pshaffer@example.net  390-747-3088   
1998          999  Gregory Duarte    caitlindunlap@example.org  257-409-8196   
1999         1000     Jason Walls        michael42@example.com  692-333-6880   

                                       

In [278]:
# Split name into two parts: First and Last
df[['First_Name', 'Last_Name']] = df['name'].str.strip().str.split(' ', n=1, expand=True)

In [279]:
print(df)

      customer_id            name                        email         phone  \
0               2     Brad Newton  taylorcatherine@example.net  537-674-1158   
1               2     Brad Newton  taylorcatherine@example.net  537-674-1158   
2               3    Larry Torres         dsanchez@example.net  810-256-4505   
3               4  Kimberly Price    jessicaknight@example.com  423-222-9779   
4               4  Kimberly Price    jessicaknight@example.com  423-222-9779   
...           ...             ...                          ...           ...   
1995          997    Jodi Simpson           eric24@example.org  483-625-2940   
1996          998   Crystal Brown         pshaffer@example.net  390-747-3088   
1997          998   Crystal Brown         pshaffer@example.net  390-747-3088   
1998          999  Gregory Duarte    caitlindunlap@example.org  257-409-8196   
1999         1000     Jason Walls        michael42@example.com  692-333-6880   

                                       

In [280]:
df.columns

Index(['customer_id', 'name', 'email', 'phone', 'address', 'registration_date',
       'loyalty_status', 'order_id', 'order_date', 'order_amount',
       'order_status', 'product_category', 'First_Name', 'Last_Name'],
      dtype='object')

In [281]:
# Get the list of columns
cols = df.columns.tolist()

# Identify the index of 'name'
name_index = cols.index('name')

# Remove the new columns from the end
cols.remove('First_Name')
cols.remove('Last_Name')

# Insert them right after 'name'
cols.insert(name_index + 1, 'First_Name')
cols.insert(name_index + 2, 'Last_Name')

# Reorder dataframe
df = df[cols]

# Preview the new column order
df.head()


,customer_id,name,First_Name,Last_Name,email,phone,address,registration_date,loyalty_status,order_id,order_date,order_amount,order_status,product_category
0,2,Brad Newton,Brad,Newton,taylorcatherine@example.net,537-674-1158,"38783 Oliver Street, West Kristenborough, MT 9...",13-07-2023,Silver,209c024e-6c32-49fa-acd1-2501bd19c1cd,2023-10-30,486.53,Pending,Toys
1,2,Brad Newton,Brad,Newton,taylorcatherine@example.net,537-674-1158,"38783 Oliver Street, West Kristenborough, MT 9...",13-07-2023,Silver,3f263091-da18-48a6-8b00-c179e1b1a81a,2024-08-29,781.22,Pending,Books
2,3,Larry Torres,Larry,Torres,dsanchez@example.net,810-256-4505,"6845 Steele Turnpike, West Erikabury, UT 37487",18-08-2023,Bronze,63d7870e-617e-4f49-ae9b-b893e635e43f,2023-04-05,454.07,Pending,Toys
3,4,Kimberly Price,Kimberly,Price,jessicaknight@example.com,423-222-9779,"1631 Alexis Meadows, Lake Amanda, CA 75179",08-12-2024,Gold,cbe7b5ef-013f-4b4f-9ac4-3b510075080a,2023-08-12,259.07,Cancelled,Electronics
4,4,Kimberly Price,Kimberly,Price,jessicaknight@example.com,423-222-9779,"1631 Alexis Meadows, Lake Amanda, CA 75179",08-12-2024,Gold,21209e05-078a-402d-a450-dd310595e5f0,2024-08-27,117.03,Completed,Electronics


In [282]:
df.isnull().sum()

customer_id            0
name                   0
First_Name             0
Last_Name              0
email                106
phone                 35
address                0
registration_date      0
loyalty_status         0
order_id               0
order_date             0
order_amount           0
order_status           0
product_category       0
dtype: int64

In [283]:
#  Clean the name column
df['name'] = df['name'].str.strip().str.title()

In [284]:
df['email'] = df.apply(
    lambda row: f"{row['name'].replace(' ', '').lower()}@company.com"
    if pd.isna(row['email']) else row['email'].strip().lower(),
    axis=1
)

In [285]:
df.isnull().sum()

customer_id           0
name                  0
First_Name            0
Last_Name             0
email                 0
phone                35
address               0
registration_date     0
loyalty_status        0
order_id              0
order_date            0
order_amount          0
order_status          0
product_category      0
dtype: int64

In [286]:
# Create a dictionary for mapping
tier_map = {
    'Gold': 2,
    'Silver': 1,
    'Bronze': 0
}

# Apply mapping to create new column
df['Customer_Tier'] = df['loyalty_status'].map(tier_map)


In [287]:

# Get the list of columns
cols = df.columns.tolist()

# Move 'Customer_Tier' right after 'loyalty_status'
loyalty_idx = cols.index('loyalty_status')
cols.insert(loyalty_idx + 1, cols.pop(cols.index('Customer_Tier')))

# Reorder DataFrame
df = df[cols]


In [288]:
df.columns

Index(['customer_id', 'name', 'First_Name', 'Last_Name', 'email', 'phone',
       'address', 'registration_date', 'loyalty_status', 'Customer_Tier',
       'order_id', 'order_date', 'order_amount', 'order_status',
       'product_category'],
      dtype='object')

In [289]:
df['phone'] = df['phone'].fillna('0000000000')

In [290]:
df.isnull().sum()   



customer_id          0
name                 0
First_Name           0
Last_Name            0
email                0
phone                0
address              0
registration_date    0
loyalty_status       0
Customer_Tier        0
order_id             0
order_date           0
order_amount         0
order_status         0
product_category     0
dtype: int64

In [291]:
df['phone'] = df['phone'].astype(str)

In [292]:
df['phone'] = df['phone'].str.replace(r'\D', '', regex=True)

In [293]:
df['phone'].str.len().value_counts()

phone
10    1989
12       3
0        3
1        3
9        2
Name: count, dtype: int64

In [294]:
df.loc[df['phone'].str.len() != 10, 'phone'] = '0000000000'

In [296]:
df.columns

Index(['customer_id', 'name', 'First_Name', 'Last_Name', 'email', 'phone',
       'address', 'registration_date', 'loyalty_status', 'Customer_Tier',
       'order_id', 'order_date', 'order_amount', 'order_status',
       'product_category'],
      dtype='object')

In [ ]:
# Ensure phone numbers are in the format +1-XXXXXXXXXX
df['phone'] = '+1-' + df['phone']


In [300]:
df.to_csv('cleaned.csv', index=False)


In [301]:
df.to_sql('unified_customer_view', con=engine, if_exists='replace', index=False)

54